# Sugarscape model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from time import sleep
from IPython.display import clear_output
import math
import warnings
import statistics

# we change the DPI settings to imporove plot resolution
mpl.rcParams['figure.dpi'] = 150

# here we initiate global variables
exceedance = 0
price_tracker_element = list()

## Environment

In [ ]:
class Sugarscape():
    
    def __init__(self, n, **params):
        self.n = n
        self.params = params

        self.capacity = self.make_capacity()
        
        self.array = self.capacity.copy()
        
        if trades:
            self.capacity_spice= self.make_capacity_spice()
            self.array_spice = self.capacity_spice.copy()

        self.make_agents()
        
        if government:
            self.array_dispensaries = self.dispensaries()
        
    def make_capacity(self):
        """ This creates the array with the values of sugar for each cell when they are full. """
        
        # calculate the minimum distance from one the two peaks of sugar
        X, Y = np.indices((self.n, self.n))
        dist1 = np.hypot(X-14, Y-14)
        dist2 = np.hypot(X-35, Y-35)
        dist = np.minimum(dist1, dist2)
        
        bins = [21, 16, 11, 6]  # values in the capacity array are set according to dist from peak
        a = np.digitize(dist, bins)
        
        return a
    
    def make_capacity_spice(self):
        """ This creates the array with the values of spice for each cell when they are full. """
        
        #calculate the minimum distance from one the two peaks of spice
        X, Y = np.indices((self.n, self.n))
        dist1 = np.hypot(X-14, Y-35)
        dist2 = np.hypot(X-35, Y-14)
        dist = np.minimum(dist1, dist2)
        
        bins = [21, 16, 11, 6]  # cells in the capacity array are set according to dist from peak
        a = np.digitize(dist, bins)
        
        return a
        
    def make_agents(self):
        """ Creates and places the agents in the grid. """
        
        n, m = self.array.shape
        if migration: 
            n, m = 20, 20
        t = [(i, j) for i in range(n) for j in range(m)]
        locs = np.array(t)
        np.random.shuffle(locs)

        # make the number of agents required, the default is 400
        num_agents = self.params.get('num_agents', 400)
        self.agents = [Agent(locs[i], self.params) 
                       for i in range(num_agents)]

        self.draw()

        self.occupied = set(agent.loc for agent in self.agents)
    
    def dispensaries(self):
        """ This sets the location of the government's dispensaries on the map. """
        t = [(i*2+36, j*2+5) for i in range(5) for j in range(5)]
        a = np.array(t)

        t = [(i*2+5, j *2+36) for i in range(5) for j in range(5)]
        b= np.array(t)

        disp = np.vstack((a, b))
        return disp
            
        
    def grow(self, totex):
        """ This function grows back the sugar (and the spice) given the growth rate until it's back at its original capacity. """

        grow_rate = 1
        self.array_copy = self.array

        # minimum between the actual capacity of the sugar (or spice) plus the growth rate and the max capacity so it doesn't exceed the latter
        self.array = np.minimum(self.array + grow_rate, self.capacity) 
        if trades:
            self.array_spice = np.minimum (self.array_spice + grow_rate, self.capacity_spice)
        
        # replenish dispensaries
        if government:
            disp = self.array_dispensaries
            for i in disp:
                # every dispensary we distribute an equal amount of the exceedance of other agents
                self.array[i[0],i[1]] = self.array_copy[i[0],i[1]] + round(totex/len(disp))
                # we max the dspensaries at 5 
                if self.array[i[0],i[1]] > 5:
                    self.array[i[0],i[1]] = 5

    def look_and_move(self, center, vision):
        """
        This function let's the agent look vertically and horizontally, 
        find the cell with the most amount of sugar, and returns that location.
        """

        # this sections creates a shuffled array of the locations that the agent can "see".
        def make_array(d):
            a = np.array([[-d, 0], [d, 0], [0, -d], [0, d]])
            return a
        arrays = [make_array(d) for d in range(1, vision+1)]
        locs = np.vstack(arrays)
        np.random.shuffle(locs)
        
        #this makes it possible to forbid negative values 
        #and those bigger than 50, changing them in a coherent way
        locs = (locs + center) % self.n 
        locs = [tuple(loc) for loc in locs]
        
        empty_locs = [loc for loc in locs if loc not in self.occupied]  # select unoccupied cells
        
        if len(empty_locs) == 0:    # if all visible cells are occupied, stay put
            return center
        
        # return the cell with the best sugar levels
        t = [self.array[loc] for loc in empty_locs]
        i = np.argmax(t)
        return empty_locs[i]
        
    def look_and_move_trade(self, center, vision, metabolism, metabolism_spice, sugar, spice, agent):
        """
        This function let's the agent trade with the nearest neighbour that maximizes wellfare and 
        then move the agent in the cell that maximizes wellfare
        """
        
        # this section is the same of the model without trading
        def make_array(d):
            a = np.array([[-d, 0], [d, 0], [0, -d], [0, d]])
            return a
        arrays = [make_array(d) for d in range(1, vision+1)]
        locs = np.vstack(arrays)
        np.random.shuffle(locs)
        locs = (locs + center) % self.n 
        locs = [tuple(loc) for loc in locs]
        empty_locs = [loc for loc in locs if loc not in self.occupied]
        occupied_locs = [loc for loc in locs if loc in self.occupied]
        
        # calculate the MRS of the agent based on its wellfare function, endowments of resources and metabolism
        mrs_agent = self.mrs(metabolism, metabolism_spice, sugar, spice)
        wellfare_agent = self.wellfare(metabolism, metabolism_spice, sugar, spice)        

        # find available to trade agents (agent_trader) that maximizes the wellfare of the agent that is moving
        # calcuate the mtrs of the possible trader, the price of the trade, and the resulting quantities
        # after trade has take place
        posizioni = [(i, i.loc) for i in self.agents]
        agent_trader = (0,0)
        for i in posizioni:
            for l in occupied_locs:
                if i[1] == l:
                    mrs_agent_trader = self.mrs(i[0].metabolism, i[0].metabolism_spice, i[0].sugar, i[0].spice)
                    price =  math.sqrt(mrs_agent_trader * mrs_agent)
                    wellfare_trader = self.wellfare(i[0].metabolism, i[0].metabolism_spice, i[0].sugar, i[0].spice)
                    person = self.quantities(price, wellfare_trader, wellfare_agent, mrs_agent, mrs_agent_trader, \
                                             metabolism, metabolism_spice, sugar, spice,i[0].metabolism, i[0].metabolism_spice,\
                                             i[0].sugar, i[0].spice, i)
                    if person:  
                        if agent_trader[1] < person[1] :
                            agent_trader = person

        # if we found the trader that maximizes our wellfare we trade
        if agent_trader != (0,0):
            delta_sug = agent_trader[3]
            delta_sp = agent_trader[4]

            agent.sugar += delta_sug
            agent_trader[0].sugar -= delta_sug

            agent.spice += delta_sp
            agent_trader[0].spice -= delta_sp

            # this keeps track of the prices of the transactions
            price_of_trade = agent_trader[5]
            global price_tracker_element
            price_tracker_element.append(price_of_trade)

        # if all visible cells are occupied, the agent stays put and does not move
        if len(empty_locs) == 0: 
            return center
        
        # find the cell that maximizes wellfare so that agent's wellfare is maximized 
        #(if there is no cell that maximizes wellfare the agent stays put)
        t = [(self.array[loc],self.array_spice[loc],loc) for loc in empty_locs]
        wellfare_loc = [(self.wellfare(metabolism, metabolism_spice, sugar + element[0]-metabolism, spice + element[1]-metabolism_spice ), element[2]) for element in t ]
        t = [(self.array[loc],self.array_spice[loc],loc) for loc in empty_locs]
        wellfare_loc = [(self.wellfare(metabolism, metabolism_spice, sugar + element[0]-metabolism, spice + element[1]-metabolism_spice ), element[2]) for element in t ]
        location = (0,0)
        for item in wellfare_loc:
            if item[0]>location[0]:
                location = item
        if location[1] == 0:
            return center

        return location[1]
    
    def harvest(self, loc):
        """ Makes the sugar in which the agent moved equal to 0. """
        sugar = self.array[loc]
        self.array[loc] = 0
        return sugar
    
    def harvest_spice(self, loc):
        """ Makes the spice in which the agent moved equal to 0. """
        spice = self.array_spice[loc]
        self.array_spice[loc]=0
        return spice
    
    def step(self):
        """ This executes one cicle of the movement of the agents and checks if they're still alive. """

        # calculate 80 percentile of sugar wealth and initiates the variable fore the total exceedance 
        # this is for the government model only)
        perc = np.percentile(self.get_sugar(), 90)
        totex=0

        # we call every agent
        random_order = np.random.permutation(self.agents)
        for agent in random_order:
            
            # mark the current cell unoccupied
            self.occupied.remove(agent.loc)
            
            # execute one step (and get the exceedance back if we have a government)
            exceedance = agent.step(self, perc, agent)
            
            if government:
                # tracker of the total exceedance
                totex= totex + exceedance
            
            # if the agent is dead, remove from the list
            if agent.is_starving() or agent.is_old():
                self.agents.remove(agent)
            else:
                # otherwise mark its cell occupied
                self.occupied.add(agent.loc)

        # grow back sugar in cells (and dispensaries using the exceedance)
        self.grow(totex)
        return len(self.agents)

    def draw(self):
        """ Draws the simulations. """
        if trades:
            array_to_draw = np.maximum(self.array, self.array_spice)
            plt.imshow(array_to_draw, cmap='YlOrRd', vmax=9, origin='lower')    # spice and sugar have the same color
        else:
            plt.imshow(self.array, cmap='YlOrRd', vmax=9, origin='lower')
        
        plt.xticks([])
        plt.yticks([])
        
        agents = self.agents
        rows, cols = np.transpose([agent.loc for agent in agents])
        xs = cols 
        ys = rows
        plt.plot(xs, ys, '.', color='black')[0]
        
        if government:
            dispensaries = self.dispensaries()        
            rows, cols = np.transpose(dispensaries)
            xs = cols 
            ys = rows
            plt.plot(xs, ys,"+", color='green')[0]        
        
    def animate(self, frames, interval=None, step=None): 
        """ This is the function that keeps the simulation going. """

        # interval in seconds between frames #steps between frames

        # getting the stats
        num_agents = [len(self.agents)]
        sug = self.get_sugar()

        mean_sugar = [np.mean(self.get_sugar())]
        mean_met = [np.mean(self.get_metabolism())]
        mean_vision = [np.mean(self.get_vision())]
        if not trades:
            gini = [self.gini_coeff(sug)]
        else:
            sp = self.get_spice()
            tot = [sum(x) for x in zip(sug, sp)]
            gini = [self.gini_coeff(tot)]
            price_tracker = dict()

            mean_sp = [np.mean(sp)]
            mean_met_sp = [np.mean(self.get_spice_met())]

        if step is None:
            step = self.step
        plt.show()

        #for every cicle call the step function, draw the grid and save the stats
        for i in range(frames-1):

            if trades:
                global price_tracker_element
                price_tracker_element = list()

                mean_sp.append(np.mean(sp))
                mean_met_sp.append(np.mean(self.get_spice_met()))
            
            self.draw() # plots the simulation
            plt.show()
            if interval:
                sleep(interval) # stops the simulation for the seconds specified in the interval
            step() # do the cicle

            num_agents.append(len(self.agents))
            sug = self.get_sugar()

            mean_sugar.append(np.mean(self.get_sugar()))
            mean_met.append(np.mean(self.get_metabolism()))
            mean_vision.append(np.mean(self.get_vision()))

            if not trades:
                gini.append(self.gini_coeff(sug))
            else:
                sp = self.get_spice()
                tot = [sum(x) for x in zip(sug, sp)]
                gini.append(self.gini_coeff(tot))
                price_tracker[i+1] = price_tracker_element
          
            clear_output(wait=True) # refreshes the plots

        self.draw()
        plt.show()

        # returns the stats
        if trades:
            interests = [num_agents, gini, price_tracker, mean_sugar, mean_met, mean_vision, mean_sp, mean_met_sp]
        else:
            interests = [num_agents, gini, mean_sugar, mean_met, mean_vision]
        return(interests)

    def get_vision(self):
        """ Returns a list with the vision of all the agents in that moment. """
        return [agent.vision for agent in self.agents]

    def get_metabolism(self):
        """ Returns a list with the metabolism of all the agents in that moment. """
        return [agent.metabolism for agent in self.agents]

    def get_lifespan(self):
        """ Returns a list with the lifespan of all the agents in that moment. """
        return [agent.lifespan for agent in self.agents]
        
    def get_sugar(self):
        """ Returns a list with the sugar of all the agents in that moment. """
        return [agent.sugar for agent in self.agents]

    def get_spice(self):
        """ Returns a list with the spices of all the agents in that moment. """
        return [agent.spice for agent in self.agents]

    def get_spice_met(self):
        """ Returns a list with the spices of all the agents in that moment. """
        return [agent.metabolism_spice for agent in self.agents]

    def gini_coeff(self, arr):
        x = np.array(arr)
        total = 0
        for i, xi in enumerate(x[:-1], 1):
            total += np.sum(np.abs(xi - x[i:]))
        return total / (len(x)**2 * np.mean(x))

    def mrs(self, metabolism, metabolism_spice, sugar, spice):
        return (spice / metabolism_spice) / (sugar / metabolism)
      
    def wellfare(self, met, met_sp, sug, spices):
        with warnings.catch_warnings():
            warnings.simplefilter('ignore') 
            wellfare = (sug**(met/ (met + met_sp))) * (spices**(met_sp / (met + met_sp)))
        return wellfare

    def quantities(self, price, well_trad, well_agent, mrs_agent, mrs_poss_trader, ag_met, ag_met_s, ag_sug, ag_sp, tr_met, tr_met_s, tr_sug, tr_sp, i):
        """ Receives characteristics and returns the outcome of the exchange if it is vantagious for both parties. """
      
        if mrs_agent >= mrs_poss_trader:
            # agent buys sugar, poss_trad buys spice
            if price > 1:
                change_sug_ag = 1
                change_sp_ag = -price
                # agent exchanges p units of spice for 1 unit of sugar
                ag_sp -= price
                tr_sp += price
                ag_sug += 1
                tr_sug -= 1
            else:
                change_sug_ag = 1/price
                change_sp_ag = -1
                # agent exchanges 1 unit of spice for 1/p unit of sugar
                ag_sp -= 1
                tr_sp += 1
                ag_sug += 1/price
                tr_sug -= 1/price
        else:
            # agent buys spice, poss_trad buys sugar
            if price >= 1:
                change_sug_ag = -1
                change_sp_ag = price
                # agent exchanges 1 units of sugar for p unit of spice
                ag_sp += price
                tr_sp -= price
                ag_sug -= 1
                tr_sug += 1
            else:
                change_sug_ag = -1/price
                change_sp_ag = +1
                # agent exchanges 1/p units of sugar for 1 unit of spice
                ag_sp += 1
                tr_sp -= 1
                ag_sug -= 1/price
                tr_sug += 1/price
      
        #calculate the new levels of wellfare
        wellfare_agent = self.wellfare(ag_met, ag_met_s, ag_sug, ag_sp)
        wellfare_trader = self.wellfare(tr_met, tr_met_s, tr_sug, tr_sp)

        if wellfare_agent >= well_agent and wellfare_trader > well_trad:
            agent=(i[0], wellfare_agent, price, change_sug_ag, change_sp_ag, price)
            return agent

## Agents

In [ ]:
class Agent:
    def __init__(self, loc, max_vision = 12, max_metabolism = 5  , min_lifespan = 600, max_lifespan = 1000 , min_sugar = 5 , max_sugar = 25):
        self.loc = tuple(loc)
        self.age = 0
        self.vision = np.random.randint(1, 13)   # max vision is set to 12
        self.metabolism = np.random.randint(1, max_metabolism+1)
        self.lifespan = np.random.randint(min_lifespan, max_lifespan+1)
        self.sugar = np.random.randint(min_sugar, max_sugar+1)

        if trades:
          self.spice = np.random.randint(min_sugar, max_sugar+1)
          self.metabolism_spice = np.random.randint(1, max_metabolism+1)


    def step(self, env, perc = 0, agent=0):
        
        # we get the new location and calculate the new sugar and the spice 
        if trades:
          self.loc = env.look_and_move_trade(self.loc, self.vision, self.metabolism, self.metabolism_spice, self.sugar, self.spice, agent) 
          self.spice += env.harvest_spice(self.loc) - self.metabolism_spice
        else:
          self.loc = env.look_and_move(self.loc, self.vision)
        self.sugar += env.harvest(self.loc) - self.metabolism # eat
        
        # we make the agents older
        self.age += 1 
        
        # we remove the excess sugar from the richer agents
        if government:
            if self.sugar > perc:
                exceedance = self.sugar - perc
                exceedance = round(exceedance)
                self.sugar = perc
                return exceedance
        return 0
        
    def is_starving(self):
        if trades:
            return self.sugar <= 0 or self.spice <=0
        else:
            return self.sugar <= 0
    
    def is_old(self):
        return self.age > self.lifespan

## Run the model

In [ ]:
trades = True
government = False
migration = False

# create initial grid and place agents
env = Sugarscape(50, num_agents=400)

In [ ]:
env.step()
list_num_agents = env.animate(frames=100,  interval = 0)

## Graphs

In [ ]:
if not trades:
  fig, axes = plt.subplots(2, 2, figsize=(15,15))
else:
  fig, axes = plt.subplots(4, 2, figsize=(15,15))

  end_sugar = env.get_sugar()
  end_spice = env.get_spice()

  met_sug = env.get_metabolism()
  met_spice = env.get_spice_met()

  wellfare_tot = []
  for i in range(len(end_sugar)):
    wellfare_tot.append((end_sugar[i]**(met_sug[i]/ (met_sug[i] + met_spice[i]))) * (end_spice[i]**(met_spice[i] / (met_sug[i] + met_spice[i]))))

  count, bins_count = np.histogram(env.get_spice())
  pdf = count / sum(count)
  cdf = np.cumsum(count)
  axes[2, 0].plot(cdf, label="CDF")
  axes[2, 0].set_title('spice')


  count, bins_count = np.histogram(wellfare_tot)
  pdf = count / sum(count)
  cdf = np.cumsum(count)
  axes[2, 1].plot(cdf, label="CDF")
  axes[2, 1].set_title('wellfare')

  count, bins_count = np.histogram(env.get_spice_met())
  pdf = count / sum(count)
  cdf = np.cumsum(count)
  axes[3, 0].plot(cdf, label="CDF")
  axes[3, 0].set_title('spice metabolism')


  sug = env.get_sugar()
  sp = env.get_spice()
  endowment = [sum(x) for x in zip(sug, sp)]
  count, bins_count = np.histogram(wellfare_tot)
  pdf = count / sum(count)
  cdf = np.cumsum(count)
  axes[3, 1].plot(cdf, label="CDF")
  axes[3, 1].set_title('total endowments')

count, bins_count = np.histogram(env.get_sugar())
cdf = np.cumsum(count)
axes[0, 0].plot(cdf, label="CDF")
axes[0, 0].set_title('sugar')

count, bins_count = np.histogram(env.get_vision())
pdf = count / sum(count)
cdf = np.cumsum(count)
axes[0, 1].plot(cdf, label="CDF")
axes[0, 1].set_title('vision')

count, bins_count = np.histogram(env.get_metabolism())
pdf = count / sum(count)
cdf = np.cumsum(count)
axes[1, 0].plot(cdf, label="CDF")
axes[1, 0].set_title('metabolism')

count, bins_count = np.histogram(env.get_lifespan())
pdf = count / sum(count)
cdf = np.cumsum(count)
axes[1, 1].plot(cdf, label="CDF")
axes[1, 1].set_title('lifespan')

In [ ]:
if not trades:
  fig, axes = plt.subplots(2, 2, figsize=(15,15)) 
else:
  fig, axes = plt.subplots(4, 2, figsize=(15,15)) 
  axes[2, 0].hist(env.get_sugar())
  axes[2, 0].set_title('spice')


  axes[2, 1].hist(wellfare_tot)
  axes[2, 1].set_title('wellfare')

  axes[3, 0].hist(env.get_spice_met())
  axes[3, 0].set_title('spice metabolism')

  axes[3, 1].hist(endowment)
  axes[3, 1].set_title('total endowments')

axes[0, 0].hist(env.get_sugar())
axes[0, 0].set_title('sugar')

axes[0, 1].hist(env.get_vision())
axes[0, 1].set_title('vision')

axes[1, 0].hist(env.get_metabolism())
axes[1, 0].set_title('metabolism')

axes[1, 1].hist(env.get_lifespan())
axes[1, 1].set_title('lifespan')

In [ ]:
if not trades:
  fig, axes = plt.subplots(2, 2, figsize=(15,15))
else:
  fig, axes = plt.subplots(4, 2, figsize=(15,15))

  end_sugar = env.get_sugar()
  end_spice = env.get_spice()

  met_sug = env.get_metabolism()
  met_spice = env.get_spice_met()

  wellfare_tot = []
  for i in range(len(end_sugar)):
    wellfare_tot.append((end_sugar[i]**(met_sug[i]/ (met_sug[i] + met_spice[i]))) * (end_spice[i]**(met_spice[i] / (met_sug[i] + met_spice[i]))))

  count, bins_count = np.histogram(env.get_spice())
  pdf = count / sum(count)
  cdf = np.cumsum(count)
  axes[2, 0].plot(cdf, label="CDF")
  axes[2, 0].set_title('spice')


  count, bins_count = np.histogram(wellfare_tot)
  pdf = count / sum(count)
  cdf = np.cumsum(count)
  axes[2, 1].plot(cdf, label="CDF")
  axes[2, 1].set_title('wellfare')

  count, bins_count = np.histogram(env.get_spice_met())
  pdf = count / sum(count)
  cdf = np.cumsum(count)
  axes[3, 0].plot(cdf, label="CDF")
  axes[3, 0].set_title('spice metabolism')


  sug = env.get_sugar()
  sp = env.get_spice()
  endowment = [sum(x) for x in zip(sug, sp)]
  count, bins_count = np.histogram(wellfare_tot)
  pdf = count / sum(count)
  cdf = np.cumsum(count)
  axes[3, 1].plot(cdf, label="CDF")
  axes[3, 1].set_title('total endowments')

count, bins_count = np.histogram(env.get_sugar())
cdf = np.cumsum(count)
axes[0, 0].plot(cdf, label="CDF")
axes[0, 0].set_title('sugar')

count, bins_count = np.histogram(env.get_vision())
pdf = count / sum(count)
cdf = np.cumsum(count)
axes[0, 1].plot(cdf, label="CDF")
axes[0, 1].set_title('vision')

count, bins_count = np.histogram(env.get_metabolism())
pdf = count / sum(count)
cdf = np.cumsum(count)
axes[1, 0].plot(cdf, label="CDF")
axes[1, 0].set_title('metabolism')

count, bins_count = np.histogram(env.get_lifespan())
pdf = count / sum(count)
cdf = np.cumsum(count)
axes[1, 1].plot(cdf, label="CDF")
axes[1, 1].set_title('lifespan')

In [ ]:
if not trades:
  fig, axes = plt.subplots(2, 2, figsize=(15,15)) 
else:
  fig, axes = plt.subplots(4, 2, figsize=(15,15)) 
  axes[2, 0].hist(env.get_sugar())
  axes[2, 0].set_title('spice')


  axes[2, 1].hist(wellfare_tot)
  axes[2, 1].set_title('wellfare')

  axes[3, 0].hist(env.get_spice_met())
  axes[3, 0].set_title('spice metabolism')

  axes[3, 1].hist(endowment)
  axes[3, 1].set_title('total endowments')

axes[0, 0].hist(env.get_sugar())
axes[0, 0].set_title('sugar')

axes[0, 1].hist(env.get_vision())
axes[0, 1].set_title('vision')

axes[1, 0].hist(env.get_metabolism())
axes[1, 0].set_title('metabolism')

axes[1, 1].hist(env.get_lifespan())
axes[1, 1].set_title('lifespan')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize = (10,10))

ax1.plot(list_num_agents[0])
ax1.set_title("Number of agents from the beginning to the end")

ax2.plot(list_num_agents[1])
ax2.set_title("Gini coefficient from the beginning to the end")

In [ ]:
if trades:
  x = []
  y = []
  n_trades = []
  variance = []
  mean = []



  for i in list(list_num_agents[2].keys()):
    n_trades.append(len(list_num_agents[2][i]))
    mean.append(statistics.mean(list_num_agents[2][i]))
    variance.append(statistics.variance(list_num_agents[2][i], mean[-1]))

    for c in list_num_agents[2][i]:
      x.append(i)
      y.append(c)



  fig, (ax0, ax1, ax2, ax3) = plt.subplots(4, figsize=(15,15))

  ax0.scatter(x, y, marker='1')
  ax0.set_title("Plot every trade's price per cicle")

  ax1.plot(n_trades)
  ax1.set_title('number of trades per cicle')

  ax2.plot(mean)
  ax2.set_title('mean price of trades per cicle')

  ax3.plot(variance)
  ax3.set_title('variance of trades per cicle')

In [ ]:
if not trades:
  fig, (ax0, ax1, ax2) = plt.subplots(3, figsize=(15,15))

  ax0.plot(list_num_agents[2])
  ax0.set_title("Mean sugar per cicle")

  ax1.plot(list_num_agents[3])
  ax1.set_title('Mean metabolism per cicle')

  ax2.plot(list_num_agents[4])
  ax2.set_title('Mean vision per cicle')

else:
  fig, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(5, figsize=(15,15))

  ax0.plot(list_num_agents[3])
  ax0.set_title("Mean sugar per cicle")

  ax1.plot(list_num_agents[4])
  ax1.set_title('Mean metabolism per cicle')

  ax2.plot(list_num_agents[5])
  ax2.set_title('Mean vision per cicle')

  ax3.plot(list_num_agents[6])
  ax3.set_title('Mean spice per cicle')

  ax4.plot(list_num_agents[7])
  ax4.set_title(' Mean spice metabolism per cicle')